## espn ff roster & transactions

- description: self-contained notebook, pulls ff roster details and transactions from espn api
- requirements: `requests` `pandas` `numpy` `json` `openpyxl`
- input: espn user credentials and league id
- output: csv files containing roster details and transactions

resources: https://github.com/cwendt94/espn-api/tree/master, https://stmorse.github.io/journal/espn-fantasy-projections.html

instructions: clone onto google colab. run notebook cells in order. download output files from google drive to wherever.

### libaries, constants, & user input

In [5]:
# libraries

import requests
import pandas as pd
import numpy as np
import json



In [7]:
# user input

# retrieve input from chrome
swid      = ''
espn_s2   = ''
league_id = 
season    = 2022
start_week = 1
end_week = 8

run_date = '20221027'


# league url
url = f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/FFL/seasons/{season}/segments/0/leagues/{league_id}'



In [8]:
# constants
# could be moved to a constants.py file sometime

# position codes
slot_dict = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

# team owner codes
team_dict = {
    0: 'free agent',
    1: 'decker',
    2: 'bard',
    3: 'billy',
    4: 'adam',
    5: 'amac',
    6: 'brendan',
    7: 'zach',
    8: 'dan',
    9: 'jono',
    10: 'nate',
    11: 'rob',
    12: 'pealer',
    13: 'fishy',
    14: 'justin'
}

# nfl team codes
pro_team_dict = {
    0 : 'None',
    1 : 'ATL',
    2 : 'BUF',
    3 : 'CHI',
    4 : 'CIN',
    5 : 'CLE',
    6 : 'DAL',
    7 : 'DEN',
    8 : 'DET',
    9 : 'GB',
    10: 'TEN',
    11: 'IND',
    12: 'KC',
    13: 'LV',
    14: 'LAR',
    15: 'MIA',
    16: 'MIN',
    17: 'NE',
    18: 'NO',
    19: 'NYG',
    20: 'NYJ',
    21: 'PHI',
    22: 'ARI',
    23: 'PIT',
    24: 'LAC',
    25: 'SF',
    26: 'SEA',
    27: 'TB',
    28: 'WSH',
    29: 'CAR',
    30: 'JAX',
    33: 'BAL',
    34: 'HOU'
}

### player data

In [11]:
# get player info

# force pull to include up to limit # of players
# otherwise, espn default pull includes only a fraction of players
filters = {
  "players": {
    "limit": 10000,
    "sortDraftRanks": {
      "sortPriority": 100,
      "sortAsc": True,
      "value": "STANDARD"
    }
  }
} 
headers = {'x-fantasy-filter': json.dumps(filters)}
        
# fetch player data
player_req = requests.get(url + '?view=kona_player_info',
                params={'scoringPeriodId': 0},
                cookies={"SWID": swid, "espn_s2": espn_s2},
                headers=headers)
player_json = player_req.json()

# run through each player and append keeper info to list
# make dataframe from combined list
player_list = []
player_dict = {}

for player in player_json['players']:
    player_name = player['player']['fullName']
    player_id = player['id']
    team_id = player['onTeamId']
    keeper_value = player['keeperValue']
    future_keeper_value = player['keeperValueFuture']
    elig_slots = player['player']['eligibleSlots']
    
    pro_team_id = player['player']['proTeamId']
    
    team = team_dict[team_id]
    
    pro_team = pro_team_dict[pro_team_id]
    
    # pos assignment
    # should refer to dict sometime
    if 0 in elig_slots:
        pos = 'QB'
    elif 2 in elig_slots:
        pos = 'RB'
    elif 4 in elig_slots:
        pos = 'WR'
    elif 6 in elig_slots:
        pos = 'TE'
    elif 16 in elig_slots:
        pos = 'DEF'
    elif 17 in elig_slots:
        pos = 'K'
    else:
        pos = 'NA'  
    
    # extra rookie field (not used)
    rookie = 25 in elig_slots
      
      
    # make list and shortened dict for player info
    player_list.append([player_name,
                        player_id,
                        team_id,
                        team,
                        pos,
                        pro_team,
                        #rookie,
                        future_keeper_value])
    
    player_dict[player_id] = {'player_name':player_name,
                              'pos': pos,
                              'rookie':rookie}

# dataframe for player
player_df = pd.DataFrame(player_list, 
                    columns=['player', 'player_id','team_id','team','pos','pro_team','keeper_value'])



# display head and output
player_df.to_csv(f'player_table_{run_date}.csv', index=False)
player_df.to_excel(f'player_table_{run_date}.xlsx', index=False)
player_df.head(10)

,player,player_id,team_id,team,pos,pro_team,keeper_value
0,Jonathan Taylor,4242335,5,amac,RB,IND,100
1,Derrick Henry,3043078,5,amac,RB,TEN,14
2,Cooper Kupp,2977187,2,bard,WR,LAR,8
3,Christian McCaffrey,3117251,10,nate,RB,SF,34
4,Justin Jefferson,4262921,2,bard,WR,MIN,1
5,Ja'Marr Chase,4362628,5,amac,WR,CIN,0
6,Austin Ekeler,3068267,2,bard,RB,LAC,3
7,Najee Harris,4241457,13,fishy,RB,PIT,0
8,Joe Mixon,3116385,8,dan,RB,CIN,7
9,Dalvin Cook,3116593,4,adam,RB,MIN,10


### transaction data

In [10]:
# get transactions

print('week: ')

trans_list = []
for week in range(start_week, end_week + 1):
    print(week, end=' ')

    # fetch data
    trans_req = requests.get(url + '?view=mTransactions2',
                     params={'scoringPeriodId': week},
                     cookies={"SWID": swid, "espn_s2": espn_s2})
    trans_json = trans_req.json()
    
    # pull only executed roster & waiver transactions
    # record bid amount if an add
    
    for trans in trans_json['transactions']:
        if 'status' not in list(trans):
            continue
        if (trans['status'] != 'EXECUTED') | (trans['scoringPeriodId'] != week) | (trans['type'] not in ['ROSTER', 'WAIVER']):
            continue
        else:
            #bid = trans['bidAmount']
            team_id = trans['teamId']
            trans_id = trans['id']
            proc_date = trans.get('processDate',0)
            team = team_dict[team_id]
            cat = trans['type']
            for item in trans['items']:
                player_id = item['playerId']
                player = player_dict[player_id]['player_name']
                pos = player_dict[player_id]['pos']
                action = item['type']
                bid = np.where(action == 'ADD', trans['bidAmount'], 0)            
                trans_list.append([
                        week, trans_id, proc_date,
                        team, team_id,
                        player_id, player, pos,
                        action, bid, cat
            ])

# make df of data
trans_df = pd.DataFrame(trans_list, 
                    columns=['week', 'trans_id','proc_date',
                             'team','team_id', 
                             'player_id', 'player', 'pos',
                             'action', 'bid', 'cat'])

# filter out lineup changes
trans_df = trans_df[trans_df['action']!='LINEUP']

# export and display
trans_df.to_csv(f'trans_table_{run_date}.csv', index=False)
trans_df.to_excel(f'trans_table_{run_date}.xlsx', index=False)
trans_df.head()

week: 
1 2 3 4 5 6 7 8 

,week,trans_id,proc_date,team,team_id,player_id,player,pos,action,bid,cat
15,1,cf413be8-49ce-4b7b-88fd-2f22b42dec7c,0,rob,11,3924327,Drew Lock,QB,DROP,0,ROSTER
25,1,7ccba5a3-6628-4275-be63-84056cfb021f,1661356806957,zach,7,2998565,Mo Alie-Cox,TE,ADD,1,WAIVER
26,1,871c5d88-46c0-498b-9f24-36cedf92e412,1661356806955,zach,7,3043116,Demarcus Robinson,WR,ADD,2,WAIVER
42,1,8b6c98be-c242-4447-bfc1-1f20e33d5d0e,1662134413425,decker,1,4372780,Tommy Tremble,TE,ADD,0,WAIVER
43,1,8b6c98be-c242-4447-bfc1-1f20e33d5d0e,1662134413425,decker,1,2972515,Cooper Rush,QB,DROP,0,WAIVER
